### What:

Trying to get node classification with deepwalk+Label Propagation to work

### TODO:

- improve documentation/comments
- add train and test split to actually test performance for classiying
    - follow the authors code for the split. See main.py in the 'classifier' folder
        - same splits and hyperparameters
    - add code for LabelPropagation() to perform the task
- compare Perozzis embeddings with karateclub
    - make their shapes compatible

In [19]:
import numpy as np
import networkx as nx
from karateclub import LabelPropagation
from sklearn.neighbors import NearestNeighbors
import embed_utils
from sklearn.model_selection import train_test_split
from copy import deepcopy
from collections import Counter

In [2]:
import random
import networkx as nx
from typing import Dict
from karateclub.estimator import Estimator


class MyLabelPropagation(Estimator):
    r"""An implementation of `"Label Propagation Clustering" <https://arxiv.org/abs/0709.2938>`_
    from the Physical Review '07 paper "Near Linear Time Algorithm to Detect Community Structures
    in Large-Scale Networks". The tool executes a series of label propagations with unique labels.
    The final labels are used as cluster memberships.
    Args:
        seed (int): Random seed. Default is 42.
        iterations (int): Propagation iterations. Default is 100.
    """

    def __init__(self, seed: int = 42, iterations: int = 7):
        self.seed = seed
        self.iterations = iterations

    def _make_a_pick(self, neighbors):
        """
        Choosing a neighbor from a propagation source node.
        Arg types:
            * **neigbours** *(list)* - Neighbouring nodes.
        """
        scores = {}
        for neighbor in neighbors:
            neighbor_label = self._labels[neighbor]
            if neighbor_label in scores.keys():
                scores[neighbor_label] = scores[neighbor_label] + 1
            else:
                scores[neighbor_label] = 1
        top = [key for key, val in scores.items() if val == max(scores.values()) and val is not None]

        if len(top) > 0:
            return random.sample(top, 1)[0]
        else:
            return None

    def _do_a_propagation(self):
        """
        Doing a propagation round.
        """
        random.shuffle(self._nodes)
        new_labels = {}
        for node in self._nodes:
            neighbors = [neb for neb in nx.neighbors(self._graph, node)]
            pick = self._make_a_pick(neighbors)
            new_labels[node] = pick
        self._labels = new_labels

    def fit(self, graph: nx.classes.graph.Graph):
        """
        Fitting a Label Propagation clustering model.
        Arg types:
            * **graph** *(NetworkX graph)* - The graph to be clustered.
        """
        self._set_seed()
        graph = self._check_graph(graph)
        self._graph = graph
        self._nodes = [node for node in self._graph.nodes()]
        self._labels = nx.get_node_attributes(self._graph, "class")
        random.seed(self.seed)
        for _ in range(self.iterations):
            self._do_a_propagation()

    def get_memberships(self) -> Dict[int, int]:
        r"""Getting the cluster membership of nodes.
        Return types:
            * **memberships** *(dict)* - Node cluster memberships.
        """
        memberships = self._labels
        return memberships

In [3]:
def k_nearest_graph(embed, original_graph, k=5):
    """"
    Construct a graph from an embedding using the k-nearest graph algorithm
    embed: embedding
    original_graph: needed for preserving node attributes
    k: nearest k neighbors to a point in the embedding space are used fix edges in the returned graph
    """
    # create new graph based on original nodes and attributes
    k_nearest_graph = nx.Graph()
    k_nearest_graph.add_nodes_from(original_graph)
    nx.set_node_attributes(k_nearest_graph, nx.get_node_attributes(original_graph, "class"),  "class")
    # find the k nearest neighbors for each node and add between the node and these neighbors
    # edges in the new graph 
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='ball_tree').fit(embed)
    _, indices = nbrs.kneighbors(embed)
    k_nearest_links = [[i[0], j] for i in indices for j in i[1:]]
    k_nearest_graph.add_edges_from(k_nearest_links)
    return k_nearest_graph

In [23]:
# get graph from data
graph_synth2 = embed_utils.data2graph("synth2")
print(f"#nodes: {len(graph_synth2.nodes())}, #edges (bidirectional): {len(graph_synth2.edges())}")

# Split into train test
train_nodes, test_nodes = train_test_split(list(graph_synth2.nodes()), train_size=.5)
# Set labels of test nodes to other classes
original_attributes = deepcopy(nx.get_node_attributes(graph_synth2, "class"))
all_attributes = nx.get_node_attributes(graph_synth2, "class")
i = max(all_attributes)
for node in test_nodes:

    all_attributes[node] = i
    i += 1

nx.set_node_attributes(graph_synth2, all_attributes, "class")

# print(all_attributes)
# print(nx.set_node_attributes(graph_synth2, all_attributes, "class"))
# print(nx.get_node_attributes(graph_synth2, "class"))
# get embedding from graph using DeepWalk
embed_synth2_karateclub = embed_utils.graph2embed(graph_synth2)
# embed_synth2_karateclub = embed_utils.load_embed("rice", "NC", method="fairwalk", implementation="singer")
print(f"shape embedding: {embed_synth2_karateclub.shape}")
labels_list = list(nx.get_node_attributes(graph_synth2, "class").values())

print(f"Counts of attributes{Counter(labels_list).most_common()}")
# construct new graph from the embedding using k-nearest graph algorithm
k_nearest_graph_synth2 = k_nearest_graph(embed_synth2_karateclub, graph_synth2)
# # use label propagation on the new graph for classification
lp = MyLabelPropagation(seed=None, iterations=7)
# lp.fit(k_nearest_graph_synth2)
lp.fit(graph_synth2)

#nodes: 500, #edges (bidirectional): 1834
shape embedding: (500, 128)
Counts of attributes[(1, 181), (0, 69), (628, 1), (533, 1), (660, 1), (644, 1), (667, 1), (738, 1), (563, 1), (622, 1), (746, 1), (687, 1), (524, 1), (681, 1), (698, 1), (525, 1), (536, 1), (650, 1), (639, 1), (600, 1), (501, 1), (715, 1), (623, 1), (664, 1), (538, 1), (571, 1), (721, 1), (540, 1), (558, 1), (503, 1), (745, 1), (648, 1), (577, 1), (626, 1), (611, 1), (633, 1), (521, 1), (700, 1), (619, 1), (526, 1), (733, 1), (520, 1), (601, 1), (617, 1), (670, 1), (638, 1), (523, 1), (630, 1), (620, 1), (690, 1), (549, 1), (722, 1), (685, 1), (530, 1), (654, 1), (705, 1), (645, 1), (665, 1), (553, 1), (731, 1), (595, 1), (597, 1), (652, 1), (691, 1), (735, 1), (578, 1), (673, 1), (637, 1), (609, 1), (747, 1), (716, 1), (748, 1), (607, 1), (505, 1), (713, 1), (568, 1), (671, 1), (542, 1), (719, 1), (567, 1), (564, 1), (513, 1), (699, 1), (742, 1), (541, 1), (634, 1), (668, 1), (625, 1), (675, 1), (686, 1), (556, 1), 

In [24]:
membs = lp.get_memberships()

correct = 0
total = len(test_nodes)
for node in test_nodes:
    t = membs[node]
    y = original_attributes[node]

    if y == t and t is not None:
        correct += 1

print(f"Accuracy is {correct/total}")

Accuracy is 0.948
